# referance :
1- https://chatgpt.com/share/679ff89b-e340-8008-ab60-8f2405df4b7a

# Dataset about `CoNLL`-2003 dataset

In [4]:
from transformers import AutoTokenizer , training_args , DataCollatorForTokenClassification , Trainer , AutoModelForTokenClassification , TrainingArguments
from accelerate import Accelerator
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from transformers import get_scheduler



In [5]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [6]:
from datasets import  load_dataset

In [7]:
raw_datasets = load_dataset("eriktks/conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [9]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [10]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [11]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [12]:
len(raw_datasets["train"][0]["ner_tags"])==len(raw_datasets["train"][0]["tokens"])

True

In [13]:
ner_feature  = raw_datasets["train"].features["ner_tags"]

In [14]:
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [15]:
ner_feature.feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

* O means the word doesn’t correspond to any entity.
* B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
* B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
* B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
* B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

# decoding the labels we saw earlier gives us this

In [110]:
ner_feature.feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [108]:
labels = raw_datasets["train"][6]["ner_tags"]

In [109]:
labels

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [16]:
words = raw_datasets["train"][1]["tokens"]
labels = raw_datasets["train"][1]["ner_tags"]

line1= " "
line2 = " "

for word , label in zip(words , labels):
  line1+= word + " "*4
  line2+= ner_feature.feature.names[label] +" "*6


In [17]:
print(line1)
print(line2)

 Peter    Blackburn    
 B-PER      I-PER      


In [18]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
tokenizer.is_fast

True

### Thats perfect the tokenizer is "fast” version available

In [20]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [21]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [119]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words= True)

In [120]:
inputs

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [1]:
inputs.tokens()

NameError: name 'inputs' is not defined

In [122]:
inputs.input_ids

[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]

In [123]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [ ]:
# ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

### in the previouse bert can dont know the lamb and partion it into la and #mb to slove this we must align all the labels with the proper words. we use -100 becuse this is ignore form loss function we use it in later (Cross Entropy)

In [98]:
def align_labels_with_tokens (labels, word_ids):
    
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(word_ids)
print(align_labels_with_tokens(labels, word_ids))

### As we can see, our function added the -100 for the two special tokens at the beginning and the end, and a new 0 for our word that was split into two tokens.

In [36]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples["tokens"],truncation = True, is_split_into_words = True
  )

  all_labels = examples["ner_tags"]
  new_labels = []
  for i , labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))
  tokenized_inputs["labels"] = new_labels
  return tokenized_inputs


In [37]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels
                                      ,batched = True
                                      ,remove_columns = raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

In [38]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [39]:
tokenized_datasets["train"][0]["labels"]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

In [40]:
raw_datasets["train"].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [41]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

### you can dont do the previouse and do the low in the later if your token align with label

In [42]:
# inputs

In [43]:
# raw_datasets["train"][0]["ner_tags"]

In [44]:
# inputs["labels"]= [raw_datasets["train"][0]["ner_tags"]]

### Fine-tuning the model with the Trainer API

### Data collation

#### We can’t just use a ``` DataCollatorWithPadding ``` because that only pads the inputs (input IDs, attention mask, and token type IDs). Here our labels should be padded the exact same way as the inputs so that they stay the same size, using -100 as a value so that the corresponding predictions are ignored in the loss computation.
### so we use DataCollatorForTokenClassification

In [45]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

### to test this for few sampples

In [33]:
batch= data_collator([tokenized_datasets["train"][i] for i in range(2)])

In [34]:
batch

{'input_ids': tensor([[  101,  7327, 19164,  2446,  2655,  2000, 17757,  2329, 12559,  1012,
           102],
        [  101,  2848, 13934,   102,     0,     0,     0,     0,     0,     0,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100]])}

In [35]:
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100]])

In [46]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8152db8d07eed2bff5a6c63705f0c4cfdf205241cb1157ae9c08f4666ebb6c5f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [47]:
import numpy as np
import evaluate

In [48]:
metric = evaluate.load("seqeval")

# but this dont take integer but take string

# test for sample

In [39]:
labels = raw_datasets["train"][0]["ner_tags"]

In [40]:
labels

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [41]:
ner_feature  = raw_datasets["train"].features["ner_tags"]

In [42]:
ner_feature.feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [43]:
labels = raw_datasets["train"][0]["ner_tags"]

In [44]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [ner_feature.feature.names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [45]:
predictions = labels.copy()
predictions[2] = "O"
predictions

['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [46]:
metric.compute(predictions = [predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [ ]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis = -1)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

  labels_list = [
      [ner_feature.feature.names[i] for i in label if i != -100]
      for label in labels
  ]
  predictions_list = [
      [ner_feature.feature.names[i] for i in prediction if i != -100]
      for prediction in predictions
  ]
  results = metric.compute(predictions=predictions_list, references=labels_list)
  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

In [49]:
id2label = {i: label for i , label in enumerate(ner_feature.feature.names)}
label2id = {v: k for k , v in id2label.items()}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

9

In [ ]:
# Fully Customize It?:

# training_args = TrainingArguments(
#     output_dir="./results",            # Where to save model checkpoints
#     evaluation_strategy="epoch",       # Evaluate at the end of every epoch
#     per_device_train_batch_size=16,    # Training batch size
#     per_device_eval_batch_size=16,     # Evaluation batch size
#     num_train_epochs=3,                # Number of training epochs
#     save_strategy="epoch",             # Save model at the end of each epoch
#     logging_dir="./logs"               # Directory for logs
# )

In [52]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,    # Training batch size
    per_device_eval_batch_size=16,     # Evaluation batch size
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,

)
trainer.train()

<ipython-input-106-dc4282e69d93>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a48104046 (a48104046-dfas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.223200,0.062669,0.464610,0.928180,0.619248,0.977189
2,0.046000,0.057733,0.451509,0.942164,0.610467,0.975966
3,0.027300,0.057175,0.496102,0.946750,0.651050,0.979581


TrainOutput(global_step=2634, training_loss=0.07746005800518016, metrics={'train_runtime': 568.2429, 'train_samples_per_second': 74.129, 'train_steps_per_second': 4.635, 'total_flos': 1020143109346326.0, 'train_loss': 0.07746005800518016, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/Ah7med/bert-finetuned-ner/commit/251b29481868af9d8b624ca2b2f5c806bf297951', commit_message='Training complete', commit_description='', oid='251b29481868af9d8b624ca2b2f5c806bf297951', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ah7med/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='Ah7med/bert-finetuned-ner'), pr_revision=None, pr_num=None)

In [62]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 14041
})

# A custom training loop using Pytorch

In [53]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle =True,
    collate_fn = data_collator,
    batch_size =8,

)

eval_dataloader= DataLoader(
    tokenized_datasets["validation"],
    collate_fn = data_collator,
    batch_size =8
)


### Next we reinstantiate our model, to make sure we’re not continuing the fine-tuning from before but starting from the BERT pretrained model again:

In [54]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
model.config.num_labels

9

In [56]:
optimizer =AdamW(model.parameters(),lr=2e-5)

In [57]:
accelerator= Accelerator()

In [58]:
model,optimizer,train_dataloader,eval_dataloader = accelerator.prepare(
    model,optimizer,train_dataloader,eval_dataloader
)

In [59]:
num_epochs =3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0 ,
    num_training_steps=num_training_steps
)

## to push our model to the Hub

In [82]:
from huggingface_hub import get_full_repo_name ,Repository

model_name = "bert-finetuned-ner-accelerate_Pt"

repo_name = get_full_repo_name(model_name)
repo_name

'Ah7med/bert-finetuned-ner-accelerate_Pt'

In [88]:
# # to clone
output_dir = "bert-finetuned-ner-accelerate_Pt"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Ah7med/bert-finetuned-ner-accelerate_Pt into local empty directory.


In [ ]:
# trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Ah7med/bert-finetuned-ner/commit/251b29481868af9d8b624ca2b2f5c806bf297951', commit_message='End of training', commit_description='', oid='251b29481868af9d8b624ca2b2f5c806bf297951', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ah7med/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='Ah7med/bert-finetuned-ner'), pr_revision=None, pr_num=None)

# Training loop
### We are now ready to write the full training loop. To simplify its evaluation part, we define this postprocess() function that takes predictions and labels and converts them to lists of strings, like our metric object expects:

In [78]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[ner_feature.feature.names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [ner_feature.feature.names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions


# training loop

In [85]:
from tqdm.auto import tqdm


In [94]:
progress_bar = tqdm(range(num_training_steps)) 

  0%|          | 0/5268 [00:00<?, ?it/s]

In [95]:
for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )
 # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )
       

epoch 0: {'precision': 0.9442890703658128, 'recall': 0.9359130724027054, 'f1': 0.9400824145227754, 'accuracy': 0.9858452348801373}
epoch 1: {'precision': 0.9442890703658128, 'recall': 0.9359130724027054, 'f1': 0.9400824145227754, 'accuracy': 0.9858452348801373}
epoch 2: {'precision': 0.9442890703658128, 'recall': 0.9359130724027054, 'f1': 0.9400824145227754, 'accuracy': 0.9858452348801373}


# Save model and Uplaod()

In [97]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/kaggle/working/bert-finetuned-ner-accelerate_Pt"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.9972959,
  'word': 'sy',
  'start': 11,
  'end': 13},
 {'entity_group': 'PER',
  'score': 0.99775606,
  'word': '##lva',
  'start': 13,
  'end': 16},
 {'entity_group': 'PER',
  'score': 0.9968732,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.6719701,
  'word': 'hugging face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9968034,
  'word': 'brooklyn',
  'start': 49,
  'end': 57}]